In [27]:
import os
import json
import base64
import boto3
import requests
import fal_client
import openai
import smtplib
from datetime import datetime
from base64 import b64decode
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from dotenv import load_dotenv, find_dotenv

In [ ]:
# Load env vars
load_dotenv(find_dotenv(), override=True)

for key in ["FINE_TUNE_ID"]:
    print(f"{key}: {os.getenv(key)}")

openai.api_key = os.getenv("OPENAI_API_KEY")
FAL_KEY = os.getenv("FAL_KEY")
FINE_TUNE_ID = os.getenv("FINE_TUNE_ID")
s3 = boto3.client('s3')
bucket_name = "falaipostings"
EMAIL_FROM = os.getenv("EMAIL_SENDER")
EMAIL_TO = os.getenv("EMAIL_RECEIVER")
smtp_user = os.getenv("SES_SMTP_USER")
smtp_pass = os.getenv("SES_SMTP_PASS")
SMTP_SERVER = os.getenv("SMTP_SERVER")
SMTP_PORT = os.getenv("SMTP_PORT")

FINE_TUNE_ID: 7580067a-4326-454c-a66d-ac2cc2a7dde4


In [29]:
def generate_post_content(prompt: str = None) -> str:
    prompt = prompt or (
        "Your are a bot specialized in creating marketing content "
        "in 5 points with emoticons and hashtags for LinkedIn posts "
        "for Baseone, a company focused on Agentic AI and analytics."
    )
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=300
    )
    return response['choices'][0]['message']['content']

In [30]:
def generate_image(prompt: str) -> str:
    result = fal_client.subscribe(
        "fal-ai/flux-pro/v1.1-ultra-finetuned",
        arguments={
            "prompt": prompt,
            "guidance_scale": 10,
            "seed": 0,
            "sync_mode": False,
            "num_images": 3,
            "enable_safety_checker": False,
            "safety_tolerance": 6,
            "output_format": "jpeg",
            "aspect_ratio": "1:1",
            "finetune_id": FINE_TUNE_ID,
            "finetune_strength": 0.7
        },
        with_logs=True,
    )
    if result and 'images' in result and result['images']:
        return result['images'][0]['url']
    return None

In [31]:
def upload_post_to_s3(post_text: str, image_url: str, bucket_name: str = bucket_name) -> str:
    """Uploads the post text + image URL to S3."""
    timestamp = datetime.utcnow().isoformat().replace(":", "-")
    filename = f"pending/post_{timestamp}.json"

    post_data = {
        "text": post_text,
        "image_url": image_url,
        "status": "pending"
    }

    s3.put_object(Bucket=bucket_name, Key=filename, Body=json.dumps(post_data))
    print(f"Uploaded post to S3 at {filename}")
    return filename

In [32]:
def upload_post_to_s3(post_text: str, image_url: str) -> str:
    timestamp = datetime.utcnow().isoformat().replace(":", "-")
    key = f"pending/post_{timestamp}.json"

    response = requests.get(image_url)
    if response.status_code != 200:
        raise Exception("Failed to download image.")

    image_base64 = base64.b64encode(response.content).decode('utf-8')
    image_data_uri = f"data:image/jpeg;base64,{image_base64}"

    post_data = {
        "text": post_text,
        "image_url": image_url,
        "image_base64": image_data_uri,
        "status": "pending"
    }

    s3.put_object(Bucket=bucket_name, Key=key, Body=json.dumps(post_data))
    print(f"Uploaded post to S3: {key}")

    send_email_to_receiver(post_text, key)
    return key

In [33]:
def send_email_to_receiver(post_text: str, key: str):
    msg = MIMEMultipart("related")
    msg["Subject"] = "New LinkedIn Post Pending Approval"
    msg["From"] = EMAIL_FROM
    msg["To"] = EMAIL_TO

    response = s3.get_object(Bucket=bucket_name, Key=key)
    post_data = json.loads(response["Body"].read())

    base64_image_data = post_data.get("image_base64", "")
    image_bytes = b64decode(base64_image_data.split(",")[1]) if "," in base64_image_data else b""

    approve_link = f"https://ther7oaeqc.execute-api.ap-south-1.amazonaws.com/default/postai?file={key}&action=approve"
    reject_link = f"https://ther7oaeqc.execute-api.ap-south-1.amazonaws.com/default/postai?file={key}&action=reject"

    html = f"""
    <html>
        <body>
            <h2>New LinkedIn Post Ready</h2>
            <p>{post_text}</p>
            <img src="cid:postimage" style="max-width: 500px; border: 1px solid #ccc;" />
            <br><br>
            <a href='{approve_link}' style='padding:10px 20px; background-color:green; color:white; text-decoration:none;'>OK</a>
            &nbsp;&nbsp;
            <a href='{reject_link}' style='padding:10px 20px; background-color:red; color:white; text-decoration:none;'>Not OK</a>
        </body>
    </html>
    """

    msg.attach(MIMEText(html, "html"))

    if image_bytes:
        img = MIMEImage(image_bytes)
        img.add_header("Content-ID", "<postimage>")
        img.add_header("Content-Disposition", "inline", filename="post.jpg")
        msg.attach(img)

    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(smtp_user, smtp_pass)
            server.sendmail(EMAIL_FROM, EMAIL_TO, msg.as_string())
            print("Email sent successfully!")
    except Exception as e:
        print(f"Email failed: {e}")

In [34]:
if __name__ == "__main__":
    print("Generating post content...")
    post_text = generate_post_content()

    print("Generating image...")
    image_prompt = "a group of SaaS analytics professionals in presentatoin screen with Baseone Name in it"
    image_url = generate_image(image_prompt)

    print("Uploading post to S3 and sending email...")
    upload_post_to_s3(post_text, image_url)

Generating post content...
Generating image...
Uploading post to S3 and sending email...
Uploaded post to S3: pending/post_2025-04-25T10-59-13.966090.json
Email sent successfully!
